In [ ]:
import numpy as np
from typing import List, Tuple

class LogisticRegression:
    def __init__(self, learning_rate=0.01, iterations=1000):
        self.lr = learning_rate
        self.iterations = iterations
        self.weights = None
        self.bias = None

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0

        for _ in range(self.iterations):
            linear_pred = np.dot(X, self.weights) + self.bias
            predictions = self.sigmoid(linear_pred)

            dw = (1 / n_samples) * np.dot(X.T, (predictions - y))
            db = (1 / n_samples) * np.sum(predictions - y)

            self.weights -= self.lr * dw
            self.bias -= self.lr * db

    def predict(self, X):
        linear_pred = np.dot(X, self.weights) + self.bias
        y_pred = self.sigmoid(linear_pred)
        return (y_pred > 0.5).astype(int)

class BaggingClassifier:
    def __init__(self, base_estimator, n_estimators=10):
        self.base_estimator = base_estimator
        self.n_estimators = n_estimators
        self.estimators = []

    def fit(self, X, y):
        n_samples = X.shape[0]
        for _ in range(self.n_estimators):
            estimator = self.base_estimator()
            indices = np.random.choice(n_samples, n_samples, replace=True)
            X_sample, y_sample = X[indices], y[indices]
            estimator.fit(X_sample, y_sample)
            self.estimators.append(estimator)

    def predict(self, X):
        predictions = np.array([est.predict(X) for est in self.estimators])
        return np.round(np.mean(predictions, axis=0)).astype(int)

class StackingClassifier:
    def __init__(self, base_estimators: List[Tuple[str, object]], final_estimator: object):
        self.base_estimators = base_estimators
        self.final_estimator = final_estimator

    def fit(self, X, y):
        # Fit base estimators
        for name, estimator in self.base_estimators:
            estimator.fit(X, y)

        # Generate meta-features
        meta_features = self._get_meta_features(X)

        # Fit final estimator
        self.final_estimator.fit(meta_features, y)

    def predict(self, X):
        meta_features = self._get_meta_features(X)
        return self.final_estimator.predict(meta_features)

    def _get_meta_features(self, X):
        return np.column_stack([est.predict(X) for _, est in self.base_estimators])

# Helper function to split data
def train_test_split(X, y, test_size=0.2):
    n_samples = X.shape[0]
    n_test = int(n_samples * test_size)
    indices = np.random.permutation(n_samples)
    test_indices = indices[:n_test]
    train_indices = indices[n_test:]
    return X[train_indices], X[test_indices], y[train_indices], y[test_indices]

# Helper function to calculate accuracy
def accuracy_score(y_true, y_pred):
    return np.mean(y_true == y_pred)

# Generate a simple dataset
np.random.seed(42)
X = np.random.randn(1000, 20)
y = (X[:, 0] + X[:, 1] > 0).astype(int)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y)

# Logistic Regression with Bagging
bagging_lr = BaggingClassifier(LogisticRegression, n_estimators=10)
bagging_lr.fit(X_train, y_train)
bagging_pred = bagging_lr.predict(X_test)

# Logistic Regression with Stacking
base_models = [
    ('lr1', LogisticRegression()),
    ('lr2', LogisticRegression()),
    ('lr3', LogisticRegression())
]
stacking_lr = StackingClassifier(base_models, LogisticRegression())
stacking_lr.fit(X_train, y_train)
stacking_pred = stacking_lr.predict(X_test)

# Simple Logistic Regression
lr = LogisticRegression()
lr.fit(X_train, y_train)
lr_pred = lr.predict(X_test)

# Print results
print("Bagging Accuracy:", accuracy_score(y_test, bagging_pred))
print("Stacking Accuracy:", accuracy_score(y_test, stacking_pred))
print("Simple Logistic Regression Accuracy:", accuracy_score(y_test, lr_pred))